# Introduction

Continual learning for semantic segmentation is a field that has emerged recently and is quickly evolving. As a newcomer to the field, I wanted to quickly get in touched with the popular approaches and see by myself some concrete results. However, if one wants to rerun the experiments made in recent papers, which often use medium- to large-scale datasets such as Pascal-VOC, ADE20K or COCO, it would require several hours of training. 

Therefore, I took inspiration from the [Simple Deep Learning project](https://https://awaywithideas.com/mnist-extended-a-dataset-for-semantic-segmentation-and-object-detection/) by Luke Tonin in which he built MNIST-Extended, a semantic segmentation dataset  made from MNIST. While this is obviously a toy dataset, it has the benefit of giving quick feedback while tinkering with models. 

I adapted it to the class-incremental scenario such that the segmentation masks are only available for the current classes. Learning the 0-9 digits is therefore performed in successive tasks. In this tutorial, we explore a scenario in which there are 2 new classes to learn at each task: (0, 1), (2, 3), (4, 5), (6, 7), (8, 9). 

This will also enable to explore common challenges of continual semantic segmentation such as Catastrophic forgetting, background shift, and the various combinations of setups where past/future classes are/are not in images and labeled as background. 

# TODO
1. 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# some initial imports
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
import sys
from tqdm import tqdm
import pandas as pd

os.chdir("/content/gdrive/MyDrive/Colab Notebooks/mnist_continual_seg")
sys.path.append("/content/gdrive/MyDrive/Colab Notebooks/simple_deep_learning")

Now that the segmentation dataset is loaded, we need to adapt it to reflect the continual setup where classes are seen sequentially. To do so, we will implement a processing function that only keeps the groundtruth masks for the current learning step. For instance, if we divide the 10 classes from MNIST in 5 learning steps of 2 classes, then at step #0 the model is only trained to segment digits 0 and 1, while other digits on the image are labeled as background (i.e. label '0'). Then, at step #1 where the model must learn to segment 2's and 3's, digits 0-1 and 4-9 will be labeled as background, and so on.

This scenario reflects the common experimental setup in which segmentation masks are only available for current classes while objects that belong to past and future classes can still appear in scene images (e.g. from Pascal-VOC) but are labeled as background. This is a challenge specific to Continual Semantic Segmentation known as Background shift, which is addressed in several recent works (MiB, PLOP, SSUL, RECALL, etc). 

In [ ]:
from trainer import Trainer, GeneticTrainer
from metrics import EvaluaterCallback
from scenarios import ContinualMnist

In [ ]:
def increment_task(scenario, trainer, evaluater, memory):
  scenario.next_task()
  task_id = scenario.train_data.curr_task_id
  trainer.next_task(scenario.n_classes_per_task[:task_id+1])
  optimizer = torch.optim.Adam(lr = 0.0005, params=trainer.model.parameters())
  trainer.set_optim(optimizer)
  evaluater.set_model(trainer.model)
  trainer.set_callbacks([evaluater])
  if memory is not None:
    memory.refactor_memory(scenario.n_classes_per_task[task_id])

def meta_train(n_tasks,
               epochs, 
               scenario,
               trainer,
               evaluater,
               memory=None, 
               pass_first_step=False,
               animation_path="animation"):
  def _print_header(scenario):
    print("*******")
    print(f"Task #{t}")
    print("*******")
    print("Classes to learn:")
    print(*[c-1 for c in scenario.train_data.curr_classes])
    print("*******")
  def _print_results():
    print()
    #evaluater.print_metrics()
    res = evaluater.metrics[-1].get_results()
    print("Overall stats")
    df = pd.DataFrame([res["Overall Acc"], res["Mean Acc"], res["Mean IoU"]]).T
    df.columns = ["Overall Acc", "Mean Acc", "Mean IoU"]
    print(df.to_markdown())

    print("Class IoU")
    df = pd.DataFrame(res["Class IoU"].values()).T
    df.columns = np.arange(-1,len(res["Class IoU"].values())-1)
    print(df.to_markdown())
    #print(tabulate(df))

    print("Class Acc")
    df = pd.DataFrame(res["Class Acc"].values()).T
    df.columns = np.arange(-1,len(res["Class IoU"].values())-1)
    print(df.to_markdown())

  def _print_new_task():
    print()
    print("####################################")
    print("Next Task")
    print("####################################")

  sample_memory=False
  
  for t in range(n_tasks):
    _print_header(scenario)
    if t > 0 :
      sample_memory = True
    
    if t > 0 or not pass_first_step:
      for i in tqdm(range(epochs)):
        trainer.train(i, scenario, memory, sample_memory=sample_memory)

    _print_results()
    
    torch.save(model.state_dict(), f"checkpoints/task-{trainer.curr_task}.pth")

    if t < n_tasks:
      _print_new_task()    
      increment_task(scenario = continual_mnist, trainer = trainer, evaluater = evaluater, memory = memory)
  evaluater.create_animation(animation_path, sample_freq=4)

  

# MiB

In [ ]:
from trainer import Trainer_MIB

from models import simple_seg_model
# Initialize dataloader, optimizer and trainer
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

#continual_mnist = ContinualMnist(n_train=5000, n_test=1000, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["confusion_matrix"], callback_frequency="epoch", n_classes=11, save_matrices=True)

trainer = Trainer_MIB(model,
                  n_classes=[3],
                  optim=optimizer,
                  from_new_class = 0,
                  lambda_distill=10,
                  output_level_distill=True,
                  callbacks=[evaluater])

In [ ]:
#Note : pour sauvegarder les prints dans un fichier
#import contextlib
 
#file_path = 'randomfile.txt'
#with open(file_path, "w") as o:
#    with contextlib.redirect_stdout(o):
#         ...

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 200,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           animation_path="MIB_lambda10_ntrain1000_epochs200")

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 200/200 [01:44<00:00,  1.92it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |       0.89405 |   0.243863 |   0.210895 |
Class IoU
|    |       -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.894402 | 0.738467 | 0.686976 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.998633 | 0.840868 | 0.842992 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 200/200 [02:14<00:00,  1.49it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.913745 |   0.397583 |   0.321261 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.920274 | 0.753494 | 0.678715 | 0.558667 | 0.622723 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.994799 | 0.852952 | 0.873763 | 0.810375 | 0.841526 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.928437 |   0.532362 |   0.420645 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.941383 | 0.763588 | 0.677026 | 0.528168 | 0.635499 | 0.562312 | 0.519119 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.991605 | 0.876598 | 0.858221 | 0.802104 | 0.834886 | 0.734751 | 0.757823 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 200/200 [02:25<00:00,  1.37it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.942401 |   0.647118 |   0.512025 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|
|  0 | 0.961577 | 0.742829 | 0.679711 | 0.501317 | 0.604219 | 0.530167 | 0.443087 | 0.659746 | 0.509625 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |       6 |        7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|--------:|---------:|----:|----:|
|  0 | 0.989138 | 0.833897 | 0.876752 | 0.794608 | 0.816494 | 0.709775 | 0.628064 | 0.78814 | 0.681432 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #4
*******
Clas

100%|██████████| 200/200 [02:34<00:00,  1.30it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.955656 |   0.762215 |   0.614365 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.976599 | 0.747526 | 0.672763 | 0.499215 | 0.580726 | 0.518667 | 0.456888 | 0.624244 | 0.516687 | 0.588354 | 0.576348 |
Class Acc
|    |       -1 |        0 |        1 |       2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.985981 | 0.847753 | 0.854359 | 0.75613 | 0.770421 | 0.654953 | 0.632928 | 0.754229 | 0.631669 | 0.763732 | 0.732213 |

####################################
Next Task
#####

#Experiments for Fine-tuning model (naive approach)

In [ ]:
from models import simple_seg_model
# Initialize dataloader, optimizer and trainer
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

#continual_mnist = ContinualMnist(n_train=5000, n_test=2500, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["confusion_matrix"], callback_frequency="epoch", n_classes=11, save_matrices=True)

trainer = Trainer(model,
                  n_classes=[3],
                  optim=optimizer,
                  callbacks=[evaluater])

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 100,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           memory=None,
           animation_path='naive')

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 100/100 [01:50<00:00,  1.10s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.894851 |   0.259316 |   0.228263 |
Class IoU
|    |       -1 |        0 |       1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|--------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.894168 | 0.829879 | 0.78685 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.998331 | 0.931362 | 0.922781 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 100/100 [01:52<00:00,  1.13s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.899462 |   0.256919 |   0.228817 |
Class IoU
|    |       -1 |   0 |   1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|----:|----:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.898811 |   0 |   0 | 0.813941 | 0.804235 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |   0 |   1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|----:|----:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.998334 |   0 |   0 | 0.908315 | 0.919464 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.893082 |   0.254253 |   0.218174 |
Class IoU
|    |       -1 |   0 |   1 |   2 |   3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|----:|----:|----:|----:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.893688 |   0 |   0 |   0 |   0 | 0.791283 | 0.714945 |   0 |   0 |   0 |   0 |
Class Acc
|    |      -1 |   0 |   1 |   2 |   3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|--------:|----:|----:|----:|----:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.99863 |   0 |   0 |   0 |   0 | 0.893317 | 0.904833 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 100/100 [02:05<00:00,  1.25s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.896283 |   0.251628 |   0.226312 |
Class IoU
|    |       -1 |   0 |   1 |   2 |   3 |   4 |   5 |        6 |        7 |   8 |   9 |
|---:|---------:|----:|----:|----:|----:|----:|----:|---------:|---------:|----:|----:|
|  0 | 0.895543 |   0 |   0 |   0 |   0 |   0 |   0 | 0.822537 | 0.771351 |   0 |   0 |
Class Acc
|    |       -1 |   0 |   1 |   2 |   3 |   4 |   5 |        6 |        7 |   8 |   9 |
|---:|---------:|----:|----:|----:|----:|----:|----:|---------:|---------:|----:|----:|
|  0 | 0.998928 |   0 |   0 |   0 |   0 |   0 |   0 | 0.896007 | 0.872969 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #4
*******
Classes to learn:
-1 8 9
*******


100%|██████████| 100/100 [02:09<00:00,  1.30s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.897374 |   0.250574 |   0.225034 |
Class IoU
|    |       -1 |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |        8 |        9 |
|---:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|---------:|---------:|
|  0 | 0.896192 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 | 0.794697 | 0.784483 |
Class Acc
|    |       -1 |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |        8 |       9 |
|---:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|---------:|--------:|
|  0 | 0.998136 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 | 0.869043 | 0.88914 |

####################################
Next Task
####################################


#Experiments with Memory model

In [ ]:
from models.memory import memory

In [ ]:
from models import simple_seg_model
# Initialize dataloader, optimizer and trainer
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

#continual_mnist = ContinualMnist(n_train=1000, n_test=500, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["acc", "iou", "confusion_matrix"], callback_frequency="step", n_classes=11, save_matrices=True)

mem = memory(images_shape=(1,60,60),
             masks_shape=(60,60),
             n_classes = 2,
             batch_size=72,
             memory_size=1000)

trainer = Trainer(model,
                  n_classes=[3],
                  optim=optimizer,
                  callbacks=[evaluater])

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 200,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           memory=mem,
           animation_path="memory_100e_72b_72bm")

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 200/200 [03:13<00:00,  1.04it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.894772 |   0.245599 |   0.212156 |
Class IoU
|    |      -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|--------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.89533 | 0.731385 | 0.706998 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.998337 | 0.888966 | 0.814283 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 200/200 [03:48<00:00,  1.14s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.904283 |   0.294001 |   0.275561 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.903418 | 0.663515 | 0.547239 | 0.432992 | 0.484005 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |       2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|--------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.999185 | 0.686422 | 0.564824 | 0.46302 | 0.520563 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.903197 |   0.302314 |   0.270565 |
Class IoU
|    |      -1 |        0 |        1 |        2 |        3 |      4 |        5 |   6 |   7 |   8 |   9 |
|---:|--------:|---------:|---------:|---------:|---------:|-------:|---------:|----:|----:|----:|----:|
|  0 | 0.90572 | 0.410376 | 0.140271 | 0.276468 | 0.180908 | 0.5391 | 0.523378 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |       4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|--------:|---------:|----:|----:|----:|----:|
|  0 | 0.998396 | 0.413438 | 0.140731 | 0.315535 | 0.192333 | 0.59892 | 0.666106 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 200/200 [03:58<00:00,  1.19s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |       0.89475 |   0.226963 |   0.204347 |
Class IoU
|    |       -1 |         0 |         1 |          2 |         3 |           4 |         5 |        6 |        7 |   8 |   9 |
|---:|---------:|----------:|----------:|-----------:|----------:|------------:|----------:|---------:|---------:|----:|----:|
|  0 | 0.895059 | 0.0583491 | 0.0037377 | 0.00485458 | 0.0448504 | 0.000892061 | 0.0727346 | 0.617062 | 0.550281 |   0 |   0 |
Class Acc
|    |      -1 |         0 |          1 |          2 |         3 |           4 |         5 |        6 |        7 |   8 |   9 |
|---:|--------:|----------:|-----------:|-----------:|----------:|------------:|----------:|---------:|---------:|----:|----:|
|  0 | 0.99906 | 0.0584551 | 0.00373795 | 0.00486978 | 0.0449805 | 0.000892061 | 0.0740401 | 0.707892 | 0.602661 |   0 |   0 |

####################################
Next Task
########

100%|██████████| 200/200 [04:02<00:00,  1.21s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.889065 |   0.185208 |   0.167559 |
Class IoU
|    |       -1 |          0 |           1 |   2 |   3 |   4 |   5 |          6 |   7 |       8 |        9 |
|---:|---------:|-----------:|------------:|----:|----:|----:|----:|-----------:|----:|--------:|---------:|
|  0 | 0.889022 | 0.00339271 | 0.000131156 |   0 |   0 |   0 |   0 | 0.00116352 |   0 | 0.48455 | 0.464889 |
Class Acc
|    |       -1 |          0 |           1 |   2 |   3 |   4 |   5 |          6 |   7 |        8 |        9 |
|---:|---------:|-----------:|------------:|----:|----:|----:|----:|-----------:|----:|---------:|---------:|
|  0 | 0.999227 | 0.00340215 | 0.000131156 |   0 |   0 |   0 |   0 | 0.00116361 |   0 | 0.534506 | 0.498853 |

####################################
Next Task
####################################


#Distillation
##Output-level distillation loss

In [ ]:
from trainer import Trainer_distillation
from models import simple_seg_model
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

#continual_mnist = ContinualMnist(n_train=1000, n_test=2500, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["acc", "iou", "confusion_matrix"], callback_frequency="epoch", n_classes=11, save_matrices=True)

trainer = Trainer_distillation(model,
                  n_classes=[3],
                  optim=optimizer,
                  from_new_class = 0,
                  lambda_distill=10,
                  output_level_distill=True,
                  callbacks=[evaluater])

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 200,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           animation_path="output_level_distillation_ntrain1000_epochs200")

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 200/200 [02:07<00:00,  1.57it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.894412 |   0.247434 |   0.214011 |
Class IoU
|    |     -1 |       0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|-------:|--------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.8947 | 0.74952 | 0.709898 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |      -1 |        0 |        1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|--------:|---------:|---------:|----:|----:|----:|----:|----:|----:|----:|----:|
|  0 | 0.99844 | 0.876479 | 0.846854 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 200/200 [02:35<00:00,  1.28it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.912543 |    0.41009 |   0.306697 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.921226 | 0.653957 | 0.543379 | 0.608836 | 0.646263 |   0 |   0 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |   4 |   5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|----:|----:|
|  0 | 0.991564 | 0.965972 | 0.929861 | 0.783015 | 0.840573 |   0 |   0 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.914876 |   0.530105 |   0.345723 |
Class IoU
|    |       -1 |        0 |        1 |        2 |       3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|--------:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.940224 | 0.534311 | 0.353908 | 0.507774 | 0.54206 | 0.553021 | 0.371651 |   0 |   0 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |   6 |   7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|----:|----:|
|  0 | 0.976776 | 0.960781 | 0.923147 | 0.685835 | 0.757347 | 0.786832 | 0.740435 |   0 |   0 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 200/200 [02:46<00:00,  1.20it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.903578 |   0.610519 |   0.371481 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|
|  0 | 0.939065 | 0.452634 | 0.241741 | 0.421116 | 0.442867 | 0.472735 | 0.348091 | 0.325382 | 0.442657 |   0 |   0 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |   8 |   9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----:|----:|
|  0 | 0.950211 | 0.942517 | 0.852067 | 0.631199 | 0.766809 | 0.581782 | 0.638165 | 0.679823 | 0.673136 |   0 |   0 |

####################################
Next Task
####################################
*******
Task #4
*******
C

100%|██████████| 200/200 [02:54<00:00,  1.15it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.872645 |   0.633999 |   0.371681 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.909889 | 0.346948 | 0.149207 | 0.370701 | 0.388957 | 0.447003 | 0.305551 | 0.297299 | 0.427482 | 0.283542 | 0.161911 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |       6 |        7 |        8 |       9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|--------:|---------:|---------:|--------:|
|  0 | 0.910172 | 0.939652 | 0.812931 | 0.574078 | 0.743245 | 0.550547 | 0.463692 | 0.50313 | 0.553697 | 0.648458 | 0.27439 |

####################################
Next Task
########

## Intermediate-level + out distillation

In [ ]:
from trainer import Trainer_distillation
from models import simple_seg_model
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

continual_mnist = ContinualMnist(n_train=1000, n_test=500, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["acc", "iou", "confusion_matrix"], callback_frequency="step", n_classes=11, save_matrices=True)

trainer = Trainer_distillation(model,
                  n_classes=[3],
                  optim=optimizer,
                  from_new_class = 0,
                  lambda_distill=10,
                  output_level_distill=True,
                  encoder_level_distill=True,
                  callbacks=[evaluater])

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 200,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           animation_path="distillation_intermediate")

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 200/200 [01:55<00:00,  1.73it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.988017 |   0.900923 |   0.841821 |
Class IoU
|    |       -1 |        0 |        1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.987756 | 0.785254 | 0.752453 | X   | X   | X   | X   | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |       1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|--------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.994692 | 0.866687 | 0.84139 | X   | X   | X   | X   | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 200/200 [02:53<00:00,  1.15it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.970977 |   0.877968 |   0.702618 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|---------:|:----|:----|:----|:----|:----|:----|
|  0 | 0.973779 | 0.744079 | 0.611172 | 0.567803 | 0.616258 | X   | X   | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|---------:|:----|:----|:----|:----|:----|:----|
|  0 | 0.980218 | 0.922697 | 0.915696 | 0.772303 | 0.798925 | X   | X   | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.935503 |   0.808642 |   0.519026 |
Class IoU
|    |       -1 |       0 |        1 |        2 |       3 |        4 |        5 | 6   | 7   | 8   | 9   |
|---:|---------:|--------:|---------:|---------:|--------:|---------:|---------:|:----|:----|:----|:----|
|  0 | 0.945914 | 0.62079 | 0.451053 | 0.422709 | 0.40278 | 0.436113 | 0.353825 | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|:----|:----|:----|:----|
|  0 | 0.949047 | 0.926183 | 0.887534 | 0.715054 | 0.736735 | 0.797086 | 0.648855 | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 200/200 [03:28<00:00,  1.04s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |       0.90356 |   0.760509 |    0.44988 |
Class IoU
|    |       -1 |       0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.920647 | 0.51977 | 0.366764 | 0.357445 | 0.296221 | 0.392499 | 0.301857 | 0.466215 | 0.427498 | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |       2 |        3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|---------:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.921527 | 0.908652 | 0.827248 | 0.65136 | 0.723426 | 0.652097 | 0.649786 | 0.767645 | 0.742845 | X   | X   |

####################################
Next Task
####################################
*******
Task #4
*******
Classes

100%|██████████| 200/200 [03:34<00:00,  1.07s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.872544 |   0.672709 |   0.384537 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.903896 | 0.404296 | 0.249562 | 0.311565 | 0.246773 | 0.368496 | 0.263955 | 0.433936 | 0.430866 | 0.282451 | 0.334106 |
Class Acc
|    |       -1 |        0 |        1 |        2 |       3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.904136 | 0.904821 | 0.843158 | 0.577177 | 0.57341 | 0.581793 | 0.545429 | 0.634699 | 0.680189 | 0.585898 | 0.569094 |

####################################
Next Task
#####

## Intermediate-level + decoder-level + output distillation

In [ ]:
from trainer import Trainer_distillation
from models import simple_seg_model
_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}
_offline = {0: list(np.arange(10))}

model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())

continual_mnist = ContinualMnist(n_train=1000, n_test=72*2, batch_size=72, tasks=_tasks)

evaluater = EvaluaterCallback(model, ["acc", "iou", "confusion_matrix"], callback_frequency="step", n_classes=11, save_matrices=True)

trainer = Trainer_distillation(model,
                  n_classes=[3],
                  optim=optimizer,
                  from_new_class = 0,
                  lambda_distill=1.25,
                  output_level_distill=True,
                  encoder_level_distill=False,
                  decoder_level_distill=True,
                  callbacks=[evaluater])

In [ ]:
meta_train(n_tasks = len(_tasks),
           epochs = 200,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           animation_path="distillation_encoder_decoder_output")

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


100%|██████████| 200/200 [00:49<00:00,  4.01it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.985297 |   0.839015 |    0.80395 |
Class IoU
|    |      -1 |        0 |        1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|--------:|---------:|---------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.98483 | 0.704047 | 0.722973 | X   | X   | X   | X   | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.996656 | 0.755472 | 0.764916 | X   | X   | X   | X   | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #1
*******
Classes to learn:
-1 2 3
*******


100%|██████████| 200/200 [01:30<00:00,  2.21it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.962843 |   0.578546 |   0.539237 |
Class IoU
|    |       -1 |        0 |        1 |        2 |       3 | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|--------:|:----|:----|:----|:----|:----|:----|
|  0 | 0.963308 | 0.409522 | 0.335609 | 0.470765 | 0.51698 | X   | X   | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |        2 |       3 | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|--------:|:----|:----|:----|:----|:----|:----|
|  0 | 0.995584 | 0.419876 | 0.337916 | 0.530095 | 0.60926 | X   | X   | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #2
*******
Classes to learn:
-1 4 5
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.938827 |   0.381739 |   0.301692 |
Class IoU
|    |       -1 |        0 |        1 |          2 |          3 |        4 |        5 | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|-----------:|-----------:|---------:|---------:|:----|:----|:----|:----|
|  0 | 0.946514 | 0.126115 | 0.110989 | 0.00724871 | 0.00320602 | 0.532312 | 0.385459 | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |          2 |          3 |        4 |        5 | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|-----------:|-----------:|---------:|---------:|:----|:----|:----|:----|
|  0 | 0.994572 | 0.127473 | 0.111081 | 0.00726158 | 0.00320736 | 0.747604 | 0.680975 | X   | X   | X   | X   |

####################################
Next Task
####################################
*******
Task #3
*******
Classes to learn:
-1 6 7
*******


100%|██████████| 200/200 [02:00<00:00,  1.66it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |       0.92779 |   0.389293 |   0.321474 |
Class IoU
|    |       -1 |        0 |         1 |   2 |   3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|---------:|----------:|----:|----:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.935002 | 0.118656 | 0.0816383 |   0 |   0 | 0.316998 | 0.286344 | 0.631016 | 0.523607 | X   | X   |
Class Acc
|    |       -1 |        0 |         1 |   2 |   3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|---------:|----------:|----:|----:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.994709 | 0.119045 | 0.0816607 |   0 |   0 | 0.336829 | 0.390513 | 0.800251 | 0.780633 | X   | X   |

####################################
Next Task
####################################
*******
Task #4
*******
Classes to learn:
-1 8 9
*******


100%|██████████| 200/200 [01:48<00:00,  1.84it/s]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.899932 |   0.275167 |   0.234491 |
Class IoU
|    |       -1 |         0 |          1 |   2 |   3 |         4 |         5 |        6 |        7 |       8 |        9 |
|---:|---------:|----------:|-----------:|----:|----:|----------:|----------:|---------:|---------:|--------:|---------:|
|  0 | 0.902696 | 0.0161027 | 0.00219962 |   0 |   0 | 0.0027115 | 0.0148982 | 0.251196 | 0.245983 | 0.58952 | 0.554093 |
Class Acc
|    |       -1 |         0 |          1 |   2 |   3 |          4 |         5 |        6 |        7 |       8 |        9 |
|---:|---------:|----------:|-----------:|----:|----:|-----------:|----------:|---------:|---------:|--------:|---------:|
|  0 | 0.997193 | 0.0161536 | 0.00219962 |   0 |   0 | 0.00271199 | 0.0150326 | 0.257586 | 0.259404 | 0.74646 | 0.730091 |

####################################
Next Task
###################################